# 패키지 Import

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup

In [2]:
# 최신 크롬 드라이버 사용하도록 세팅: 현재 OS에 설치된 크롬 브라우저 버전에 맞게 cache에 드라이버 설치
from selenium.webdriver.chrome.service import Service
service = Service(ChromeDriverManager().install())

In [7]:
import time
import random
import pandas as pd

# 무한 스크롤 함수
## 1) 기능
- 콘텐츠 로딩을 충분히 기다리며 스크롤이 불가할 때까지 스크롤을 무한 반복하는 함수

## 2) 역할
- 유튜브 웹 페이지는 스크롤을 해야 새로운 콘텐츠 정보를 제공하기 때문에, 모든 검색 결과를 확인하기 위해서는 무한 스크롤 기능 필요

In [9]:
def scroll():
    try:        
        # 페이지 내 스크롤 높이 받아오기
        last_page_height = driver.execute_script("return document.documentElement.scrollHeight")
        while True:
            # 임의의 페이지 로딩 시간 설정
            # PC환경에 따라 로딩시간 최적화를 통해 scraping 시간 단축 가능
            pause_time = random.uniform(1, 2)
            # 페이지 최하단까지 스크롤
            driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
            # 페이지 로딩 대기
            time.sleep(pause_time)
            # 무한 스크롤 동작을 위해 살짝 위로 스크롤(i.e., 페이지를 위로 올렸다가 내리는 제스쳐)
            driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight-50)")
            time.sleep(pause_time)
            # 페이지 내 스크롤 높이 새롭게 받아오기
            new_page_height = driver.execute_script("return document.documentElement.scrollHeight")
            # 스크롤을 완료한 경우(더이상 페이지 높이 변화가 없는 경우)
            if new_page_height == last_page_height:
                print("스크롤 완료")
                break
                
            # 스크롤 완료하지 않은 경우, 최하단까지 스크롤
            else:
                last_page_height = new_page_height
            
    except Exception as e:
        print("에러 발생: ", e)

# 데이터 Scrap

In [10]:
# 검색 키워드 설정: 키워드 내 띄어쓰기는 URL에서 '+'로 표시되기 때문에 이에 맞게 변환
SEARCH_KEYWORD = '연남동 맛집'.replace(' ', '+')

In [22]:
driver = webdriver.Chrome(service=service)
# 스크래핑 할 URL 세팅
URL = "https://www.youtube.com/results?search_query=" + SEARCH_KEYWORD
# 크롬 드라이버를 통해 지정한 URL의 웹 페이지 오픈
driver.get(URL)
# 웹 페이지 로딩 대기
time.sleep(3)
# 무한 스크롤 함수 실행
scroll()

스크롤 완료


In [23]:
# 페이지 소스 추출
html_source = driver.page_source
soup_source = BeautifulSoup(html_source, 'html.parser')

# 데이터 추출

In [24]:
# 콘텐츠 제목만 추출
content_total_title = list(map(lambda data: data.get_text().replace("\n", ""), content_total))
# 콘텐츠 링크만 추출
content_total_link = list(map(lambda data: "https://youtube.com" + data["href"], content_total))

# 조회수 & 업로드 날짜 추출
content_record_src = soup_source.find_all(class_ = 'style-scope ytd-video-meta-block')
content_view_cnt = [content_record_src[i].get_text().replace('조회수 ', '') for i in range(5, len(content_record_src), 10)]
content_upload_date = [content_record_src[i].get_text() for i in range(6, len(content_record_src), 10)]

# Ensure all lists are the same length
target_length = len(content_total_title)
min_length = min(len(content_total_title), len(content_total_link), len(content_view_cnt), len(content_upload_date))
content_total_title = content_total_title[:target_length]
content_total_link = content_total_link[:target_length]
content_view_cnt = content_view_cnt[:target_length]
content_upload_date = content_upload_date[:target_length]

# 딕셔너리 포맷팅
content_total_dict = {
    'title': content_total_title, 
    'link': content_total_link, 
    'view': content_view_cnt,
    'upload_date': content_upload_date
}

# Create DataFrame
df = pd.DataFrame(content_total_dict)
df.to_csv("../data/youtube_shorts", encoding='utf-8-sig', index=False)


In [33]:
# 데이터 추출

# 모든 비디오 요소를 찾음
videos = driver.find_elements(By.CSS_SELECTOR, 'ytd-video-renderer, ytd-grid-video-renderer')

# shorts만 필터링
shorts = [video for video in videos if 'shorts' in video.find_element(By.CSS_SELECTOR, 'a').get_attribute('href')]

# shorts 콘텐츠 제목과 링크 추출
shorts_titles = [short.find_element(By.CSS_SELECTOR, '#video-title').text for short in shorts]
shorts_links = [short.find_element(By.CSS_SELECTOR, 'a').get_attribute('href') for short in shorts]

# shorts의 조회수와 업로드 날짜 추출
content_view_cnt = []
content_upload_date = []
content_video_pk = []

for short in shorts:
    meta_block = short.find_element(By.CSS_SELECTOR, 'ytd-video-meta-block')
    view_count = meta_block.find_element(By.XPATH, './/span[contains(text(), "조회수")]').text.replace('조회수 ', '')
    upload_date = meta_block.find_element(By.XPATH, './/span[contains(text(), "전")]').text  # Assumes the date contains "전" (ago in Korean)
    content_view_cnt.append(view_count)
    content_upload_date.append(upload_date)

    video_link = short.find_element(By.CSS_SELECTOR, 'a').get_attribute('href')
    video_id = video_link.split('/')[-1]  # link의 '/'를 기준으로 slicing 해서 리스트로 넣고 마지막 원소 가져오기
    content_video_pk.append(video_id)

# 리스트 길이 맞춤
min_length = min(len(shorts_titles), len(shorts_links), len(content_view_cnt), len(content_upload_date))
shorts_titles = shorts_titles[:min_length]
shorts_links = shorts_links[:min_length]
content_view_cnt = content_view_cnt[:min_length]
content_upload_date = content_upload_date[:min_length]

# 딕셔너리 포맷팅
content_total_dict = {
    'title': shorts_titles,
    'link': shorts_links,
    'view': content_view_cnt,
    'upload_date': content_upload_date,
    'video_pk': content_video_pk
}

# 데이터프레임 저장

In [35]:
df = pd.DataFrame(content_total_dict)
df.to_csv("../data/youtube_shorts.csv", encoding='utf-8-sig')

# 데이터 확인

In [21]:
df

,title,link,view,upload_date
0,풍자 연남동 맛집 종결 | 또간집 EP.1,https://youtube.com/watch?v=13fi46HgCKw&pp=ygU...,•,
1,"연남동 맛집 top 20, 연남동 주민입니다.",https://youtube.com/watch?v=aoBJy7CN4No&pp=ygU...,,\n\n
2,연남동 경의선숲길 어떻게 돌아다니지? 핫플 맛집｜서울핫플 서울여행,https://youtube.com/watch?v=qVlzC7OUlys&pp=ygU...,\n\n,
3,길바닥에서 떠들다가 오늘 굶을 뻔했다😭 연남동에서 오후 3시반 첫 끼로 떡볶이 어때...,https://youtube.com/watch?v=51vGu9VzCh0&pp=ygU...,,\n\n\n\n\n\n\n\n\n\n\n\n\n•\n\n\n\n\n•\n조회수 6....
4,"서울 연남, 연희동 하루코스✨BEST 13✨ 👀볼거리 가득 연남 연희 맛집, 쇼핑,...",https://youtube.com/watch?v=MsbKqHnA22w&pp=ygU...,\n\n\n\n\n\n\n\n\n\n\n\n\n•\n\n\n\n\n•\n9.2천회\...,\n\n\n\n\n\n\n\n\n\n\n\n•\n\n
...,...,...,...,...
112,이번 주말엔 연남동 이 루트로 추천!◡( ๑❛ᴗ❛ )◡ 연남동 맛집 가이드📖,https://youtube.com/shorts/PLE4PeOh9y0,\n\n\n\nRIRI FOOD&TRAVEL\n\n\n \n RIRI FOO...,\n\n\nRIRI FOOD&TRAVEL\n\n\n \n RIRI FOOD&...
113,[VLOG] 연남동 맛집 추천🍽 포코리트(Pocorit),https://youtube.com/watch?v=qPpvG6VXxTY&pp=ygU...,\n\n\nMarketSquare Korea\n\n\n \n MarketSq...,•
114,진리의 치맥! 한잔하기 딱 좋은 연남동 맛집 신난닭 | 고기걸과 술맛난다,https://youtube.com/watch?v=uhpekZ8Huzs&pp=ygU...,•,
115,[미어캣] 힐링되는 정원 카페! 연남동 데이트 / 연남동 맛집 / 연남동 카페 / ...,https://youtube.com/watch?v=kbS1vbIjI-o&pp=ygU...,,\n\n•\n조회수 1.2천회\n1년 전\n\n


In [54]:
import os
from dotenv import load_dotenv
from openai import OpenAI

# API 키를 환경변수에서 가져오기
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

client = OpenAI(api_key=api_key)

def extract_store_name_from_title(title):
    response = client.completions.create(
        model="gpt-3.5-turbo-instruct",
        prompt=f"Extract the store name from the following video title: {title}",
        max_tokens=50,
        n=1,
        stop=None,
        temperature=0.5,
    )

    store_name = response.choices[0].text.strip()
    return store_name

# gpt가 추측한 식당 이름 리스트
guess_name = []

for title in shorts_titles:
    store_name = extract_store_name_from_title(title)
    guess_name.append(title)
    print(f"Video Title: {title} -> Store Name: {store_name}")





Title: 연남동 맛집 빠르고 쉽게 정리 #연남동 #연남동맛집 #연남동데이트 -> Store Name: 연남동 맛집
Title: [Day +257] 연남동 숨겨진 프토 맛집 발견!✨ -> Store Name: The store name is "숨겨진 프토 맛집 발견".
Title: 리뷰만 13,000개 넘게 달린 연남동 맛집 -> Store Name: 연남동 맛집
Title: 맛잘알 구독자들이 알려준 홍대 맛집 투어 -> Store Name: 홍대 맛집 투어
Title: #연남동데이트 #홍대맛집 #오이시쿠 #데이트맛집 #먹방 -> Store Name: 오이시쿠
Title: 연남 필수 코스 타코집... 무조건 들리세요 -> Store Name: The store name is "타코집" (Taco House).
Title: 연남동 맛집 여기 꼭가세요.. 🔥#shorts -> Store Name: 연남동 맛집
Title: 연남동 맛집 TOP5 -> Store Name: 연남동 맛집
Title: 두 그릇 같은 한 그릇이 나오는 연남동 맛집 -> Store Name: 연남동 맛집
Title: #연남물갈비 #연남물갈비홍대본점 #홍대맛집추천 #물갈비맛집 #홍대입구역맛집추천 #연남동맛집추천 #산더미물갈비 #물갈비 #맛집추천 #mukbang #먹중사 -> Store Name: 연남물갈비홍대본점
Title: 연남동 맛집 로야토야 오마카세 점심 후기 -> Store Name: 로야토야
Title: 홍대 맛집 찾지마세요 #연남타코 #연남동 #연남동맛집 #홍대맛집 #홍대 #홍대술집 #홍대데이트 #타코 #타코맛집 #파히타 #대창 #대창맛집 #대창먹방 #맛집 #맛집추천 #맛집소개 -> Store Name: 홍대 맛집
Title: 이래서 연남동 연남동 하는구나_여보야 나 오늘도 네발로 들어가 #연남동맛집 #중화복춘 #낮술 -> Store Name: 중화복춘
Title: 연남동 맛집 감나무집 기사식당 리뷰 -> Store Name: 감나무집 기사식당
Title: 연남동 줄서서먹는 맛집 

KeyboardInterrupt: 